<a href="https://colab.research.google.com/github/HarshithaTL/Find-my-funds/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install langchain faiss-cpu InstructorEmbedding openai spacy pandas
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [24]:
import pandas as pd

# Load your JSON files
mf_holdings_df = pd.read_json("mf_holdings_data.json")
mutual_funds_df = pd.read_json("mutual_funds_data.json")
stock_data_df = pd.read_json("stock_data.json")

# Preview each dataset
print("Mutual Funds:")
display(mutual_funds_df.head())

print("MF Holdings:")
display(mf_holdings_df.head())

print("Stock Data:")
display(stock_data_df.head())


ValueError: Trailing data

In [25]:
for file in ["mf_holdings_data.json", "mutual_funds_data.json", "stock_data.json"]:
    print(f"\nPreviewing: {file}")
    with open(file) as f:
        for _ in range(3):
            print(f.readline().strip())



Previewing: mf_holdings_data.json
{"parentSchemeCode": 49135, "noShares": 4620, "marketValue": 4667.21, "aum": 41209.32, "holdingPercentage": 11.3256176030083, "finCode": 1124516, "assetCode": 2, "srNo": 2, "rating": "CARE AAA", "investmentDate": "2025-03-31", "asset": "Debt", "assetType": "Corporate Debt", "sector": "Others", "category": "Small Cap", "parentInternalSecurityId": 3655363995, "parentIsin": "INF174KA1SX6"}
{"parentSchemeCode": 49135, "noShares": 4620, "marketValue": 4667.21, "aum": 41209.32, "holdingPercentage": 11.3256176030083, "finCode": 1124516, "assetCode": 2, "srNo": 2, "rating": "CARE AAA", "investmentDate": "2025-03-31", "asset": "Debt", "assetType": "Corporate Debt", "sector": "Others", "category": "Small Cap", "parentInternalSecurityId": 4281304498, "parentIsin": "INF174KA1SY4"}
{"parentSchemeCode": 49168, "noShares": 3800, "marketValue": 3838.8322, "aum": 73346.96, "holdingPercentage": 5.23379864686962, "finCode": 1124516, "assetCode": 2, "srNo": 11, "rating":

In [4]:
import pandas as pd

# Load your JSON files, handling potential multiple JSON objects per line
try:
    mf_holdings_df = pd.read_json("mf_holdings_data.json")
    mutual_funds_df = pd.read_json("mutual_funds_data.json")
    stock_data_df = pd.read_json("stock_data.json")
except ValueError:
    # If the standard read fails, try reading with lines=True
    mf_holdings_df = pd.read_json("mf_holdings_data.json", lines=True)
    mutual_funds_df = pd.read_json("mutual_funds_data.json", lines=True)
    stock_data_df = pd.read_json("stock_data.json", lines=True)

# Preview each dataset
print("Mutual Funds:")
display(mutual_funds_df.head())

print("MF Holdings:")
display(mf_holdings_df.head())

print("Stock Data:")
display(stock_data_df.head())

Mutual Funds:


,schemeCode,amfiCode,amcCode,schemeName,typeCode,optCode,classCode,plan,etf,stp,...,maxDrawdowns,internalSecurityId,directGrowthId,assetAllocation,sizeExposure,totalHoldingPercentage,fundPrimarySector,expenseRatioDate,expenseRatio,sectorExposure
0,47641,151452.0,400027,SBI FMP-78-1170D(IDCW)-Direct Plan,2,2,21,Direct,False,False,...,"{'7D': 0.0, '1M': 0.0, '3M': -0.01565503265813...",6.334236e+08,1.193315e+09,"{'debt': 95.43, 'cash': 4.57}",{'Small Cap': 100.00000000000006},100.0,None,NaN,NaN,NaN
1,45469,NaN,400004,Aditya Birla SL FTP-TF-1098D(IDCW)-Direct Plan,2,2,21,Direct,False,False,...,NaN,1.512703e+09,1.683469e+09,NaN,NaN,NaN,None,NaN,NaN,NaN
2,41032,NaN,400004,Aditya Birla SL FTP-QZ-1100D-(D)-Direct Plan,2,2,21,Direct,False,False,...,NaN,2.645789e+09,3.100308e+09,NaN,NaN,NaN,None,NaN,NaN,NaN
3,49498,153131.0,400030,Tata BSE Select Business Groups Index Fund(IDC...,1,2,79,Direct,False,True,...,"{'7D': -7.87019236050842, '1M': -9.62357074348...",4.193622e+09,1.853941e+09,"{'equity': 100.2, 'cash': -0.2}","{'Large Cap': 95.40205298767708, 'Mid Cap': 4....",100.0,Energy,2025-03-31,0.28,"{'Materials': 20.46991712870023, 'Information ..."
4,43213,146997.0,400045,Union Overnight Fund-Reg(G),1,1,65,Regular,False,True,...,"{'7D': 0.0, '1M': 0.0, '3M': 0.0, '6M': 0.0, '...",1.054675e+09,1.683184e+09,"{'debt': 1.97, 'cash': 98.03}",{'Small Cap': 100.00000000000006},100.0,None,2025-02-28,0.18,NaN


MF Holdings:


,parentSchemeCode,noShares,marketValue,aum,holdingPercentage,finCode,assetCode,srNo,rating,investmentDate,...,parentIsin,name,shortName,ticker,isin,status,indCode,industry,internalSecurityId,securityType
0,49135,4620.0,4667.2100,41209.32,11.325618,1124516,2,2,CARE AAA,2025-03-31,...,INF174KA1SX6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49135,4620.0,4667.2100,41209.32,11.325618,1124516,2,2,CARE AAA,2025-03-31,...,INF174KA1SY4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49168,3800.0,3838.8322,73346.96,5.233799,1124516,2,11,IND AAA,2025-03-31,...,INF846K017X1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49109,4620.0,4667.2100,41209.32,11.325618,1124516,2,2,CARE AAA,2025-03-31,...,INF174KA1SW8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30692,500.0,505.1100,804330.28,0.062799,1124516,2,124,CARE AAA,2025-03-31,...,INF174K01D28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Stock Data:


,finCode,bseScripCode,bseScripName,bseScripGroup,name,industryCode,industry,hseCode,ticker,nseSeries,...,standardDeviation,volatility,1YVolatility,2YVolatility,3YVolatility,4YVolatility,5YVolatility,10YVolatility,rollingReturns,maxDrawdowns
0,233727,710045.0,JCOPOCRPS1,None,JCOPOCRPS1,113.0,Unspecified,206.0,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,292072,NaN,None,None,Sahaj Fashions Ltd.,98.0,Textile,206.0,SAHAJ,ST,...,77.106175,77.106175,77.106175,NaN,NaN,NaN,NaN,NaN,{'1Y': -60.23153161607577},"{'7D': 0.0, '1M': -38.05309734513274, '3M': -5..."
2,131168,531168.0,ASSOC CERAMI,XT,Associated Ceramics Ltd.,87.0,Refractories,206.0,None,None,...,48.285958,48.285958,48.285958,48.663341,48.716488,NaN,NaN,NaN,"{'1Y': -47.43725760882135, '2Y': 664.579592209...","{'7D': -1.9955156950672452, '1M': -13.03223239..."
3,297403,543911.0,ATALREAL,B,Atal Realtech Ltd.,29.0,Construction - Real Estate,206.0,ATALREAL,EQ,...,47.860070,47.860070,47.860070,50.785770,56.299735,62.285109,62.822652,NaN,"{'1Y': 33.39168739316894, '2Y': 18.27704911064...","{'7D': -2.542372881355946, '1M': -2.8571428571..."
4,243998,540774.0,IFGLEXPOR,B,IFGL Refractories Ltd.,87.0,Refractories,23.0,IFGLEXPOR,EQ,...,42.647393,42.647393,42.647393,50.835355,45.835997,45.544619,50.592366,NaN,"{'1Y': -45.511665987023, '2Y': 45.668482610914...","{'7D': -9.642669469259072, '1M': -9.6426694692..."


In [27]:
import json
import pandas as pd

def load_clean_json_lines(filepath):
    data = []
    with open(filepath, "r") as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue  # skip blank lines
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"[Line {i}] Skipping invalid JSON: {e}")
    return pd.DataFrame(data)

# Use this for all three files
mf_holdings_df = load_clean_json_lines("mf_holdings_data.json")
mutual_funds_df = load_clean_json_lines("mutual_funds_data.json")
stock_data_df = load_clean_json_lines("stock_data.json")


[Line 1655] Skipping invalid JSON: Expecting ':' delimiter: line 1 column 431 (char 430)
[Line 10038] Skipping invalid JSON: Expecting ',' delimiter: line 1 column 132 (char 131)
[Line 11880] Skipping invalid JSON: Expecting ':' delimiter: line 1 column 102 (char 101)
[Line 25514] Skipping invalid JSON: Expecting ',' delimiter: line 1 column 260 (char 259)
[Line 27383] Skipping invalid JSON: Expecting ':' delimiter: line 1 column 209 (char 208)
[Line 32421] Skipping invalid JSON: Expecting ',' delimiter: line 1 column 254 (char 253)
[Line 34083] Skipping invalid JSON: Expecting ',' delimiter: line 1 column 509 (char 508)
[Line 35775] Skipping invalid JSON: Expecting ':' delimiter: line 1 column 107 (char 106)
[Line 37439] Skipping invalid JSON: Expecting ',' delimiter: line 1 column 431 (char 430)
[Line 49363] Skipping invalid JSON: Expecting ':' delimiter: line 1 column 164 (char 163)
[Line 57898] Skipping invalid JSON: Expecting ',' delimiter: line 1 column 304 (char 303)
[Line 59577

In [28]:
!pip install -U langchain-community
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# Load embedding model
embedding_model = INSTRUCTOR('hkunlp/instructor-large')

def embed_texts(texts):
    instruction = "Represent the financial fund for semantic search:"
    return embedding_model.encode([[instruction, text] for text in texts])

# Prepare documents
documents = [Document(page_content=row["description"], metadata=row.to_dict()) for _, row in funds_df.iterrows()]

# Embed and store in FAISS
texts = [doc.page_content for doc in documents]
vectors = embed_texts(texts)
vectorstore = FAISS.from_embeddings(embeddings=vectors, documents=documents)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [7]:
!pip install -U InstructorEmbedding
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# Load embedding model, removing the token argument
embedding_model = INSTRUCTOR('hkunlp/instructor-large')

def embed_texts(texts):
    instruction = "Represent the financial fund for semantic search:"
    return embedding_model.encode([[instruction, text] for text in texts])

# Prepare documents, assuming funds_df is defined previously
documents = [Document(page_content=row["description"], metadata=row.to_dict())
            for _, row in mutual_funds_df.iterrows()] # Assuming funds_df should be mutual_funds_df

# Embed and store in FAISS
texts = [doc.page_content for doc in documents]
vectors = embed_texts(texts)
vectorstore = FAISS.from_embeddings(embeddings=vectors, documents=documents)

TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [8]:
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# Load embedding model, removing the token argument
embedding_model = INSTRUCTOR('hkunlp/instructor-large')

def embed_texts(texts):
    instruction = "Represent the financial fund for semantic search:"
    return embedding_model.encode([[instruction, text] for text in texts])

# Prepare documents, assuming funds_df is defined previously
documents = [Document(page_content=row["description"], metadata=row.to_dict())
            for _, row in mutual_funds_df.iterrows()] # Assuming funds_df should be mutual_funds_df

# Embed and store in FAISS
texts = [doc.page_content for doc in documents]
vectors = embed_texts(texts)
vectorstore = FAISS.from_embeddings(embeddings=vectors, documents=documents)

TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [9]:
pip install InstructorEmbedding


In [10]:
from InstructorEmbedding import INSTRUCTOR

# Correct way to initialize
embedding_model = INSTRUCTOR('hkunlp/instructor-large')

# Example embedding
instruction = "Represent the mutual fund description for retrieval"
texts = ["This mutual fund focuses on tech companies like Apple and Microsoft."]
embeddings = embedding_model.encode([[instruction, text] for text in texts])


TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [11]:
from InstructorEmbedding import INSTRUCTOR  # ✅ correct import

# Initialize the model
embedding_model = INSTRUCTOR('hkunlp/instructor-large')

# Encode example text
instruction = "Represent the mutual fund description for retrieval"
texts = ["This mutual fund focuses on tech companies like Apple and Microsoft."]
embeddings = embedding_model.encode([[instruction, text] for text in texts])

print(embeddings.shape)


TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [12]:
from InstructorEmbedding import INSTRUCTOR  # ✅ Correct import

# Initialize the model
embedding_model = INSTRUCTOR('hkunlp/instructor-large')  # ✅ No token argument

# Example usage
instruction = "Represent the mutual fund description for retrieval"
texts = ["This mutual fund focuses on tech companies like Apple and Microsoft."]
inputs = [[instruction, text] for text in texts]

embeddings = embedding_model.encode(inputs)

print(embeddings.shape)


TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [13]:
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# Load embedding model, removing the token argument
embedding_model = INSTRUCTOR('hkunlp/instructor-large')

def embed_texts(texts):
    instruction = "Represent the financial fund for semantic search:"
    return embedding_model.encode([[instruction, text] for text in texts])

# Prepare documents, assuming funds_df is defined previously
documents = [Document(page_content=row["description"], metadata=row.to_dict())
            for _, row in mutual_funds_df.iterrows()] # Assuming funds_df should be mutual_funds_df

# Embed and store in FAISS
texts = [doc.page_content for doc in documents]
vectors = embed_texts(texts)
vectorstore = FAISS.from_embeddings(embeddings=vectors, documents=documents)

TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [29]:
!pip install InstructorEmbedding faiss-cpu pandas scikit-learn transformers nltk


In [30]:
from InstructorEmbedding import INSTRUCTOR

embedding_model = INSTRUCTOR('hkunlp/instructor-large')  # Or instructor-base for faster inference


TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [31]:
!pip install -U sentence-transformers faiss-cpu



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [1]:
from sentence_transformers import SentenceTransformer

# Load a small, fast model for embedding
embedding_model = SentenceTransformer("BAAI/bge-small-en")  # Also try: all-MiniLM-L6-v2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
import pandas as pd
import json

# Replace with your actual file path if needed
with open("mutual_funds_data.json") as f:
    # Read the file line by line
    data = []
    for line in f:
        # Try to load each line as a separate JSON object
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Skipping invalid line: {line.strip()} due to error: {e}")

    # Now normalize the list of dictionaries into a DataFrame
    mutual_funds = pd.json_normalize(data)

mutual_funds.head()

Skipping invalid line: {"schemeCode": 2827, "amfiCode": 105283, "amcCode": 400027, "schemeName": "SBI Liquid Fund-Reg(FN-IDCW)", "typeCode": 1, "optCode": 2, "classCode": 24, "plan": "Regular", "etf": false, "stp": true, "primaryFund": false, "primaryFundSchemeCode": 2828, "sip": false, "swp": true, "switch": true, "minimumInvestment": 5000, "multiples": 1, "incrementalInvestment": 5000, "additionalMultiples": 1, "status": "Active", "dividendFrequency": 6, "fundManager1": 440, "fundManager2": 154, "fundManager3": null, "fundManager4": null, "name": "SBI Liquid Fund", "amfiType": "Liquid", "dividendOptionFlag": "Both", "lockIn": false, "isPurchaseAvailable": true, "minRedemptionAmount": 5000, "redemptionMultipleAmount": 1, "minSwitchAmount": 5000, "switchMultipleAmount": 1, "inceptionDate": "2007-03-22", "lockPeriod": 0, "securityType": "MUTUAL_FUNDS", "foundingYear": "2007", "riskOMeter": "Low to Moderate", "objective": "To provide the investors an opportunity to invest in the entire r

,schemeCode,amfiCode,amcCode,schemeName,typeCode,optCode,classCode,plan,etf,stp,...,sectorExposure.Others,sizeExposure.Others,schiskOMeter,schbenchmarkTier1,schemebenchmarkTier1,5YVolatilibsite,sory,amcCtinoRatio,10YVolatilitode,benchminRedemptionAmount
0,47641,151452.0,400027.0,SBI FMP-78-1170D(IDCW)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45469,NaN,400004.0,Aditya Birla SL FTP-TF-1098D(IDCW)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41032,NaN,400004.0,Aditya Birla SL FTP-QZ-1100D-(D)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49498,153131.0,400030.0,Tata BSE Select Business Groups Index Fund(IDC...,1.0,2.0,79.0,Direct,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43213,146997.0,400045.0,Union Overnight Fund-Reg(G),1.0,1.0,65.0,Regular,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def build_fund_text(row):
    name = row.get("fund_name", "")
    category = row.get("category", "")
    sector = row.get("sector", "")
    holdings = ", ".join(row.get("top_holdings", [])) if isinstance(row.get("top_holdings"), list) else ""

    return f"{name} - Category: {category}, Sector: {sector}, Holdings: {holdings}"

texts = mutual_funds.apply(build_fund_text, axis=1).tolist()


In [4]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("BAAI/bge-small-en")  # Or try: all-MiniLM-L6-v2

# Generate embeddings for real fund descriptions
mf_embeddings = embedding_model.encode(texts, show_progress_bar=True)


Batches:   0%|          | 0/993 [00:00<?, ?it/s]

In [5]:
import faiss
import numpy as np

embedding_dim = mf_embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_dim)

index.add(np.array(mf_embeddings))


In [6]:
def search_funds(user_query, k=5):
    query_embedding = embedding_model.encode([user_query])
    distances, indices = index.search(np.array(query_embedding), k)
    return mutual_funds.iloc[indices[0]]

# Example search
search_results = search_funds("funds with hdfc holdings")
search_results[["fund_name", "category", "sector"]]


KeyError: "['fund_name', 'sector'] not in index"

In [7]:
print(mutual_funds.columns.tolist())


['schemeCode', 'amfiCode', 'amcCode', 'schemeName', 'typeCode', 'optCode', 'classCode', 'plan', 'etf', 'stp', 'primaryFund', 'primaryFundSchemeCode', 'sip', 'swp', 'switch', 'minimumInvestment', 'multiples', 'incrementalInvestment', 'additionalMultiples', 'status', 'dividendFrequency', 'fundManager1', 'fundManager2', 'fundManager3', 'fundManager4', 'name', 'amfiType', 'dividendOptionFlag', 'lockIn', 'isPurchaseAvailable', 'minRedemptionAmount', 'redemptionMultipleAmount', 'minSwitchAmount', 'switchMultipleAmount', 'inceptionDate', 'lockPeriod', 'securityType', 'foundingYear', 'riskOMeter', 'objective', 'updationDateRatios', 'standardDeviation_Raw', 'sharpeRatio', 'sortinoRatio', 'aum', 'navDate', 'nav', 'price', 'benchmarkIndex', 'indexGroup', 'subGroup', 'benchmarkTier1', 'benchmarkTier2', 'isin', 'series', 'longSchemeDescription', 'shortSchemeDescription', 'schemeStatus', 'flag', 'className', 'assetType', 'category', 'subCategory', 'option', 'type', 'amcName', 'amcSymbol', 'amcWebsit

In [9]:
def build_fund_text(row):
    name = row.get("name", "")  # or whatever your actual column is
    category = row.get("category", "")
    sector = row.get("sector", "")  # or "" if not present
    holdings = ", ".join(row.get("top_holdings", [])) if isinstance(row.get("top_holdings"), list) else ""

    return f"{name} - Category: {category}, Sector: {sector}, Holdings: {holdings}"


In [10]:
# Replace with your actual column names
search_results[["name", "category"]]  # or whatever columns exist


,name,category
12,Canara Robeco ELSS Tax Saver,Equity Schemes
14,ICICI Prudential Transportation and Logistics ...,Equity Schemes
18,ICICI Prudential Value Discovery Fund,Equity Schemes
19,Bandhan Sterling Value Fund,Equity Schemes
23,Motilal Oswal Flexi Cap Fund,Equity Schemes


In [11]:
def search_funds(user_query, k=5):
    query_embedding = embedding_model.encode([user_query])
    distances, indices = index.search(np.array(query_embedding), k)
    results = mutual_funds.iloc[indices[0]].copy()
    results["similarity"] = 1 - distances[0]  # Convert L2 to similarity (optional)
    return results


In [12]:
search_results = search_funds("hdfc tax saving funds")
search_results.head()


,schemeCode,amfiCode,amcCode,schemeName,typeCode,optCode,classCode,plan,etf,stp,...,sizeExposure.Others,schiskOMeter,schbenchmarkTier1,schemebenchmarkTier1,5YVolatilibsite,sory,amcCtinoRatio,10YVolatilitode,benchminRedemptionAmount,similarity
0,47641,151452.0,400027.0,SBI FMP-78-1170D(IDCW)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.644356
1,45469,NaN,400004.0,Aditya Birla SL FTP-TF-1098D(IDCW)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.644356
2,41032,NaN,400004.0,Aditya Birla SL FTP-QZ-1100D-(D)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.644356
4,43213,146997.0,400045.0,Union Overnight Fund-Reg(G),1.0,1.0,65.0,Regular,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.644356
5,25858,NaN,400028.0,IDFC FTP-66(D)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.644356


In [13]:
import spacy
nlp = spacy.load("en_core_web_sm")



In [14]:
KEYWORDS = {
    "sector": ["tech", "technology", "infra", "infrastructure", "energy", "finance", "banking"],
    "category": ["tax", "elss", "growth", "debt", "equity"],
    "fund_house": ["sbi", "hdfc", "icici", "axis"]
}

def extract_keywords(query):
    keywords = {}
    q_lower = query.lower()

    # Rule-based keyword matching
    for key, terms in KEYWORDS.items():
        for term in terms:
            if term in q_lower:
                keywords[key] = term.capitalize()
                break  # only match one per category

    # Named Entity Recognition (NER) using spaCy
    doc = nlp(query)
    company_names = [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PRODUCT", "GPE"]]

    # Optionally clean and assign as "holding" or "company"
    if company_names:
        keywords["holding"] = company_names[0]  # Just take the first for now

    return keywords


In [15]:
query = "Find mutual funds investing in Reliance and technology sector"
print(extract_keywords(query))

# Output:
# {'sector': 'Technology', 'holding': 'Reliance'}


{'sector': 'Tech', 'holding': 'Reliance'}


In [16]:
def extract_keywords(query):
    keywords = {}
    q = query.lower()

    if "hdfc" in q:
        keywords["holding"] = "HDFC"
    if "tech" in q or "technology" in q:
        keywords["sector"] = "Technology"
    if "tax" in q:
        keywords["category"] = "ELSS"
    if "infra" in q:
        keywords["sector"] = "Infrastructure"
    if "sbi" in q:
        keywords["fund_house"] = "SBI"

    return keywords


In [17]:
def rerank_with_metadata(results, keywords):
    score = []
    for _, row in results.iterrows():
        s = 0
        if "sector" in keywords and keywords["sector"].lower() in str(row.get("sector", "")).lower():
            s += 1
        if "category" in keywords and keywords["category"].lower() in str(row.get("category", "")).lower():
            s += 1
        if "holding" in keywords and keywords["holding"].lower() in str(row.get("top_holdings", "")).lower():
            s += 1
        if "fund_house" in keywords and keywords["fund_house"].lower() in str(row.get("fund_house", "")).lower():
            s += 1
        score.append(s)

    results["metadata_score"] = score
    return results.sort_values(by=["metadata_score", "similarity"], ascending=False)


In [18]:
def full_fund_search_pipeline(user_query, k=10):
    initial_results = search_funds(user_query, k)
    keywords = extract_keywords(user_query)
    reranked_results = rerank_with_metadata(initial_results, keywords)
    return reranked_results.head(5)  # Top 5 results


In [19]:
final_results = full_fund_search_pipeline("funds with hdfc holdings in technology sector")
final_results


,schemeCode,amfiCode,amcCode,schemeName,typeCode,optCode,classCode,plan,etf,stp,...,schiskOMeter,schbenchmarkTier1,schemebenchmarkTier1,5YVolatilibsite,sory,amcCtinoRatio,10YVolatilitode,benchminRedemptionAmount,similarity,metadata_score
314,33165,134870.0,400020.0,LIC MF ULIS Regular Premium Reducing Cover HY ...,1.0,2.0,87.0,Regular,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.765754,0
328,42629,146720.0,400015.0,ICICI Pru Retirement Fund-Hybrid Aggressive Pl...,1.0,2.0,87.0,Direct,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.765754,0
426,42773,146927.0,400004.0,Aditya Birla SL Retirement Fund-40(IDCW)-Direc...,1.0,2.0,87.0,Direct,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.765754,0
593,48227,152105.0,400028.0,Bandhan Retirement Fund-Reg(IDCW),1.0,2.0,87.0,Regular,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.765754,0
594,48227,152105.0,400028.0,Bandhan Retirement Fund-Reg(IDCW),1.0,2.0,87.0,Regular,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.765754,0


In [20]:
import pandas as pd

# Load your evaluation dataset
eval_data = pd.read_csv("Dataset Queries-Securities for Model Creation.csv")

# Preview
eval_data.head()


,query,security_name
0,Is it beneficial to hold Union Overnight Fund ...,Union Overnight Fund
1,Is SBI Fixed Maturity Plan (FMP) - Series 78 (...,SBI Fixed Maturity Plan (FMP) - Series 78 (117...
2,What's the growth option NAV of UTI Money Mark...,UTI Money Market Fund
3,Can you provide details about the dividend opt...,ICICI Prudential Transportation and Logistics ...
4,What is the NAV of Motilal Oswal Nifty Microca...,Motilal Oswal Nifty Microcap 250 Index Fund


In [21]:
def evaluate_model(eval_data, k=5):
    correct = 0
    total = len(eval_data)
    missed = []

    for idx, row in eval_data.iterrows():
        query = row["Query"]
        expected = row["Correct Fund Name"].lower().strip()

        results = full_fund_search_pipeline(query, k)
        fund_names = results["name"].str.lower().tolist()

        if any(expected in fund for fund in fund_names):
            correct += 1
        else:
            missed.append({
                "query": query,
                "expected": expected,
                "returned": fund_names[:3]
            })

    accuracy = correct / total
    print(f"\n✅ Top-{k} Accuracy: {accuracy:.2%} ({correct}/{total})")

    if missed:
        print("\n❌ Missed queries:")
        for miss in missed:
            print(f"- Query: '{miss['query']}' | Expected: '{miss['expected']}' | Got: {miss['returned']}")

    return accuracy


In [22]:
evaluate_model(eval_data, k=5)


KeyError: 'Query'

In [23]:
print(eval_data.columns.tolist())


['query', 'security_name']


In [24]:
def evaluate_model(eval_data, k=5):
    correct = 0
    total = len(eval_data)
    missed = []

    for idx, row in eval_data.iterrows():
        query = row["<QUERY_COLUMN_NAME>"]  # <-- Replace this
        expected = row["<EXPECTED_FUND_COLUMN_NAME>"].lower().strip()  # <-- Replace this

        results = full_fund_search_pipeline(query, k)
        fund_names = results["name"].str.lower().tolist()

        if any(expected in fund for fund in fund_names):
            correct += 1
        else:
            missed.append({
                "query": query,
                "expected": expected,
                "returned": fund_names[:3]
            })

    accuracy = correct / total
    print(f"\n✅ Top-{k} Accuracy: {accuracy:.2%} ({correct}/{total})")

    if missed:
        print("\n❌ Missed queries:")
        for miss in missed:
            print(f"- Query: '{miss['query']}' | Expected: '{miss['expected']}' | Got: {miss['returned']}")

    return accuracy


In [25]:
print(eval_data.columns.tolist())


['query', 'security_name']


In [26]:
def evaluate_model(eval_data, k=5):
    correct = 0
    total = len(eval_data)
    missed = []

    for idx, row in eval_data.iterrows():
        query = row["query"]
        expected = row["security_name"].lower().strip()

        # Get top-k results from your model
        results = full_fund_search_pipeline(query, k)
        fund_names = results["name"].str.lower().tolist()

        if any(expected in fund for fund in fund_names):
            correct += 1
        else:
            missed.append({
                "query": query,
                "expected": expected,
                "returned": fund_names[:3]
            })

    accuracy = correct / total
    print(f"\n✅ Top-{k} Accuracy: {accuracy:.2%} ({correct}/{total})")

    if missed:
        print("\n❌ Missed queries:")
        for miss in missed:
            print(f"- Query: '{miss['query']}' | Expected: '{miss['expected']}' | Got: {miss['returned']}")

    return accuracy


In [27]:
evaluate_model(eval_data, k=5)



✅ Top-5 Accuracy: 1.06% (9/850)

❌ Missed queries:
- Query: 'Is SBI Fixed Maturity Plan (FMP) - Series 78 (1170 Days) better than other similar funds?' | Expected: 'sbi fixed maturity plan (fmp) - series 78 (1170 days)' | Got: ['lic mf unit linked insurance scheme', 'icici prudential retirement fund - hybrid aggressive plan', 'aditya birla sun life retirement fund - the 40s plan']
- Query: 'What's the growth option NAV of UTI Money Market Fund?' | Expected: 'uti money market fund' | Got: ['lic mf unit linked insurance scheme', 'icici prudential retirement fund - hybrid aggressive plan', 'aditya birla sun life retirement fund - the 40s plan']
- Query: 'Can you provide details about the dividend option in ICICI Prudential Transportation and Logistics Fund?' | Expected: 'icici prudential transportation and logistics fund' | Got: ['lic mf unit linked insurance scheme', 'icici prudential retirement fund - hybrid aggressive plan', 'aditya birla sun life retirement fund - the 40s plan']
- Qu

0.010588235294117647

In [28]:
instruction = "Represent the mutual fund based on investment theme and holdings:"


In [29]:
from sentence_transformers import SentenceTransformer

# Load the BGE model
model = SentenceTransformer("BAAI/bge-small-en")


In [30]:
formatted_query = "Represent this sentence for retrieval: " + query
query_vector = model.encode(formatted_query, normalize_embeddings=True)


In [31]:
from difflib import SequenceMatcher

def is_match(expected, fund_list):
    for fund in fund_list:
        similarity = SequenceMatcher(None, expected, fund).ratio()
        if similarity > 0.85:  # adjustable threshold
            return True
    return False


In [32]:
from difflib import SequenceMatcher

# Helper function for fuzzy match
def is_match(expected, fund_list, threshold=0.85):
    for fund in fund_list:
        similarity = SequenceMatcher(None, expected, fund).ratio()
        if similarity >= threshold:
            return True
    return False

# Evaluation function
def evaluate_model(eval_data, k=5):
    correct = 0
    total = len(eval_data)
    missed = []

    for idx, row in eval_data.iterrows():
        query = row["query"]
        expected = row["security_name"].lower().strip()  # <-- this defines `expected`

        # Get top-k results from your model
        results = full_fund_search_pipeline(query, k)
        fund_names = results["name"].str.lower().tolist()

        if is_match(expected, fund_names):
            correct += 1
        else:
            missed.append({
                "query": query,
                "expected": expected,
                "returned": fund_names[:3]
            })

    accuracy = correct / total
    print(f"\n✅ Top-{k} Accuracy: {accuracy:.2%} ({correct}/{total})")

    if missed:
        print("\n❌ Missed queries:")
        for miss in missed:
            print(f"- Query: '{miss['query']}' | Expected: '{miss['expected']}' | Got: {miss['returned']}")

    return accuracy


In [34]:
funds_df = pd.read_json("mutual_funds_data.json")
funds_df.head()


ValueError: Trailing data

In [35]:
import json
import pandas as pd

def load_json_lines_safely(filepath):
    records = []
    with open(filepath, 'r') as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                records.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping line {i}: {e}")
    return pd.DataFrame(records)

# Load the mutual funds data safely
funds_df = load_json_lines_safely("mutual_funds_data.json")
funds_df.head()


Skipping line 282: Expecting ',' delimiter: line 1 column 3421 (char 3420)
Skipping line 556: Expecting ',' delimiter: line 1 column 2149 (char 2148)
Skipping line 834: Expecting ':' delimiter: line 1 column 3358 (char 3357)
Skipping line 1108: Expecting ',' delimiter: line 1 column 2014 (char 2013)
Skipping line 1388: Expecting ':' delimiter: line 1 column 921 (char 920)
Skipping line 2230: Expecting ',' delimiter: line 1 column 2139 (char 2138)
Skipping line 2777: Extra data: line 1 column 2145 (char 2144)
Skipping line 3059: Expecting ':' delimiter: line 1 column 1519 (char 1518)
Skipping line 3613: Expecting ',' delimiter: line 1 column 3634 (char 3633)
Skipping line 3886: Expecting ',' delimiter: line 1 column 2828 (char 2827)
Skipping line 4436: Expecting ':' delimiter: line 1 column 1677 (char 1676)
Skipping line 5256: Expecting ',' delimiter: line 1 column 2413 (char 2412)
Skipping line 5537: Extra data: line 1 column 1253 (char 1252)
Skipping line 5813: Expecting ':' delimiter

,schemeCode,amfiCode,amcCode,schemeName,typeCode,optCode,classCode,plan,etf,stp,...,expenseRatio,sectorExposure,schiskOMeter,schbenchmarkTier1,schemebenchmarkTier1,5YVolatilibsite,sory,amcCtinoRatio,10YVolatilitode,benchminRedemptionAmount
0,47641,151452.0,400027.0,SBI FMP-78-1170D(IDCW)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45469,NaN,400004.0,Aditya Birla SL FTP-TF-1098D(IDCW)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41032,NaN,400004.0,Aditya Birla SL FTP-QZ-1100D-(D)-Direct Plan,2.0,2.0,21.0,Direct,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49498,153131.0,400030.0,Tata BSE Select Business Groups Index Fund(IDC...,1.0,2.0,79.0,Direct,False,True,...,0.28,"{'Materials': 20.46991712870023, 'Information ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43213,146997.0,400045.0,Union Overnight Fund-Reg(G),1.0,1.0,65.0,Regular,False,True,...,0.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
print(funds_df.columns)



Index(['schemeCode', 'amfiCode', 'amcCode', 'schemeName', 'typeCode',
       'optCode', 'classCode', 'plan', 'etf', 'stp',
       ...
       'expenseRatio', 'sectorExposure', 'schiskOMeter', 'schbenchmarkTier1',
       'schemebenchmarkTier1', '5YVolatilibsite', 'sory', 'amcCtinoRatio',
       '10YVolatilitode', 'benchminRedemptionAmount'],
      dtype='object', length=114)


In [37]:
def make_description(row):
    parts = [
        f"Fund Name: {row.get('schemeName', '')}",
        f"Plan: {row.get('plan', '')}",
        f"Class Code: {row.get('classCode', '')}",
        f"Primary Sector: {row.get('fundPrimarySector', '')}",
        f"Expense Ratio: {row.get('expenseRatio', '')}",
        f"Total Holding %: {row.get('totalHoldingPercentage', '')}",
    ]
    return ". ".join(str(p) for p in parts if p)

funds_df["description"] = funds_df.apply(make_description, axis=1)


In [41]:
import pandas as pd
import numpy as np
import faiss
import torch
import json
from sentence_transformers import SentenceTransformer

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load model
model = SentenceTransformer("BAAI/bge-small-en", device=device)

# ---- Step 1: Load JSON Lines file safely ----
def load_json_lines_safely(filepath):
    records = []
    with open(filepath, 'r') as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                records.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping line {i}: {e}")
    return pd.DataFrame(records)

# Load mutual funds data
funds_df = load_json_lines_safely("mutual_funds_data.json")

# ---- Step 2: Detect 'description' column ----
print("Available columns:", funds_df.columns.tolist())

# Try to find a description-like column
possible_cols = ["description", "objective", "details", "fund_objective", "summary"]
desc_col = next((col for col in possible_cols if col in funds_df.columns), None)

if desc_col is None:
    raise ValueError("No suitable description column found. Please check your data.")

print(f"Using '{desc_col}' as the description column.")

# ---- Step 3: Encode fund descriptions ----
fund_texts = funds_df[desc_col].fillna("").tolist()

fund_vectors = model.encode(
    fund_texts,
    batch_size=64,
    normalize_embeddings=True,
    show_progress_bar=True
)
fund_vectors = np.array(fund_vectors).astype("float32")

# ---- Step 4: Create FAISS index ----
index = faiss.IndexFlatIP(fund_vectors.shape[1])  # cosine similarity (due to normalization)
index.add(fund_vectors)

# ---- Step 5: Map index to fund data ----
fund_id_to_data = dict(enumerate(funds_df.to_dict(orient="records")))

print(f"✅ Loaded and indexed {len(fund_vectors)} mutual funds.")


Using device: cpu
Skipping line 282: Expecting ',' delimiter: line 1 column 3421 (char 3420)
Skipping line 556: Expecting ',' delimiter: line 1 column 2149 (char 2148)
Skipping line 834: Expecting ':' delimiter: line 1 column 3358 (char 3357)
Skipping line 1108: Expecting ',' delimiter: line 1 column 2014 (char 2013)
Skipping line 1388: Expecting ':' delimiter: line 1 column 921 (char 920)
Skipping line 2230: Expecting ',' delimiter: line 1 column 2139 (char 2138)
Skipping line 2777: Extra data: line 1 column 2145 (char 2144)
Skipping line 3059: Expecting ':' delimiter: line 1 column 1519 (char 1518)
Skipping line 3613: Expecting ',' delimiter: line 1 column 3634 (char 3633)
Skipping line 3886: Expecting ',' delimiter: line 1 column 2828 (char 2827)
Skipping line 4436: Expecting ':' delimiter: line 1 column 1677 (char 1676)
Skipping line 5256: Expecting ',' delimiter: line 1 column 2413 (char 2412)
Skipping line 5537: Extra data: line 1 column 1253 (char 1252)
Skipping line 5813: Expec

Batches:   0%|          | 0/497 [00:00<?, ?it/s]

✅ Loaded and indexed 31776 mutual funds.


In [42]:
def search_funds(query, k=5):
    formatted_query = "Represent this sentence for retrieval: " + query
    query_vector = model.encode(formatted_query, normalize_embeddings=True)

    D, I = index.search(np.array([query_vector]), k)
    results = [fund_id_to_data[i] for i in I[0]]
    return pd.DataFrame(results)


In [43]:
search_funds("best infra fund with low expense", k=5)


,schemeCode,amfiCode,amcCode,schemeName,typeCode,optCode,classCode,plan,etf,stp,...,expenseRatio,sectorExposure,schiskOMeter,schbenchmarkTier1,schemebenchmarkTier1,5YVolatilibsite,sory,amcCtinoRatio,10YVolatilitode,benchminRedemptionAmount
0,43399,147155.0,400056.0,ITI Liquid Fund-Reg(FN-IDCW),1.0,2.0,24.0,Regular,False,True,...,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43399,147155.0,400056.0,ITI Liquid Fund-Reg(FN-IDCW),1.0,2.0,24.0,Regular,False,True,...,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,43406,147163.0,400056.0,ITI Liquid Fund(M-IDCW)-Direct Plan,1.0,2.0,24.0,Direct,False,True,...,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,43407,147161.0,400056.0,ITI Liquid Fund(A-IDCW)-Direct Plan,1.0,2.0,24.0,Direct,False,True,...,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43407,147161.0,400056.0,ITI Liquid Fund(A-IDCW)-Direct Plan,1.0,2.0,24.0,Direct,False,True,...,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_entities(query):
    doc = nlp(query)
    return [ent.text for ent in doc.ents if ent.label_ in ["ORG", "GPE", "PRODUCT"]]


In [45]:
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_entities_and_keywords(query):
    doc = nlp(query)
    keywords = {}

    for ent in doc.ents:
        if ent.label_ in ["ORG", "PRODUCT"]:
            keywords["schemeName"] = ent.text  # could also map to top_holdings if desired
        if ent.label_ == "PERCENT" and "expense" in query.lower():
            try:
                percent_val = float(ent.text.replace("%", "").strip())
                keywords["expenseRatio"] = percent_val
            except:
                pass

    q = query.lower()

    # Soft rule-based fallback
    if "technology" in q or "tech" in q or "it" in q:
        keywords["fundPrimarySector"] = "Technology"
    if "infrastructure" in q or "infra" in q:
        keywords["fundPrimarySector"] = "Infrastructure"
    if "tax" in q:
        keywords["classCode"] = "ELSS"
    if "low expense" in q or "cheap" in q:
        keywords["expenseRatio"] = "low"

    return keywords


In [46]:
def full_fund_search_pipeline(query, k=5):
    ...
    keywords = extract_entities_and_keywords(query)
    top_funds["score_boost"] = 0  # Init score

    for key, value in keywords.items():
        if key in top_funds.columns:
            if key == "expenseRatio" and value == "low":
                top_funds["score_boost"] += -top_funds["expenseRatio"].astype(float)
            else:
                top_funds["score_boost"] += top_funds[key].astype(str).str.lower().apply(
                    lambda x: 1 if str(value).lower() in x else 0
                )

    top_funds = top_funds.sort_values("score_boost", ascending=False).head(k)
    return top_funds


In [47]:
eval_data = pd.read_csv("Dataset Queries-Securities for Model Creation.csv")
eval_data.columns  # confirm 'query' and 'security_name' are correct


Index(['query', 'security_name'], dtype='object')

In [48]:
evaluate_model(eval_data, k=5)


UnboundLocalError: cannot access local variable 'top_funds' where it is not associated with a value

In [49]:
def full_fund_search_pipeline(query, k=5):
    # Step 1: Encode the query
    formatted_query = "Represent this sentence for retrieval: " + query
    query_vector = model.encode(formatted_query, normalize_embeddings=True)

    # Step 2: FAISS search
    D, I = index.search(query_vector.reshape(1, -1), k*10)  # Retrieve more for reranking

    if I is None or len(I[0]) == 0:
        return pd.DataFrame()  # fail gracefully

    # Step 3: Retrieve top candidate funds
    top_funds = funds_df.iloc[I[0]].copy()

    # Step 4: Extract keywords from the query
    keywords = extract_entities_and_keywords(query)

    # Step 5: Initialize score_boost column
    top_funds["score_boost"] = 0

    # Step 6: Apply reranking boosts
    for key, value in keywords.items():
        if key in top_funds.columns:
            if key == "expenseRatio" and value == "low":
                # Boost by low expense
                try:
                    top_funds["expenseRatio"] = top_funds["expenseRatio"].astype(float)
                    top_funds["score_boost"] += -top_funds["expenseRatio"]
                except:
                    pass
            else:
                top_funds["score_boost"] += top_funds[key].astype(str).str.lower().apply(
                    lambda x: 1 if str(value).lower() in x else 0
                )

    # Step 7: Sort and return top-k
    top_funds = top_funds.sort_values("score_boost", ascending=False).head(k)
    return top_funds


In [50]:
evaluate_model(eval_data, k=5)


TypeError: 'float' object is not iterable

In [52]:
import difflib
import pandas as pd

def safe_str(val):
    """Converts to string safely, handling NaNs or floats."""
    if pd.isna(val):
        return ""
    return str(val)

def evaluate_model(eval_data, k=5):
    """
    Evaluates predictions against ground truth using fuzzy string matching.

    Parameters:
        eval_data (list of dict): Each dict should have keys: 'query', 'ground_truth', 'predictions'
        k (int): Top-k accuracy to check
    """
    total = len(eval_data)
    correct_topk = 0
    exact_matches = 0

    for entry in eval_data:
        query = safe_str(entry.get("query", ""))
        ground_truth = safe_str(entry.get("ground_truth", ""))
        predictions = entry.get("predictions", [])

        # Convert all predictions to strings safely
        predictions = [safe_str(p) for p in predictions]

        # Check if ground truth is in top-k predictions using fuzzy match
        match_found = False
        for pred in predictions[:k]:
            similarity = difflib.SequenceMatcher(None, ground_truth, pred).ratio()
            if similarity > 0.8:  # Adjust threshold as needed
                match_found = True
                break

        if match_found:
            correct_topk += 1

        # Optional: check if top-1 is exact match
        if predictions:
            if ground_truth.strip().lower() == predictions[0].strip().lower():
                exact_matches += 1

    topk_accuracy = correct_topk / total if total else 0
    top1_exact = exact_matches / total if total else 0

    print(f"\n🔍 Evaluation Results:")
    print(f"→ Top-{k} accuracy (fuzzy): {topk_accuracy:.2%}")
    print(f"→ Top-1 exact match:        {top1_exact:.2%}")



In [54]:
def evaluate_model(eval_data, k=5):
    """
    Evaluates predictions when each entry is a tuple: (query, ground_truth, predictions)
    """
    def safe_str(val):
        if pd.isna(val):
            return ""
        return str(val)

    total = len(eval_data)
    correct_topk = 0
    exact_matches = 0

    for entry in eval_data:
        if isinstance(entry, dict):
            query = safe_str(entry.get("query", ""))
            ground_truth = safe_str(entry.get("ground_truth", ""))
            predictions = entry.get("predictions", [])
        elif isinstance(entry, (list, tuple)) and len(entry) == 3:
            query, ground_truth, predictions = entry
            query = safe_str(query)
            ground_truth = safe_str(ground_truth)
            predictions = [safe_str(p) for p in predictions]
        else:
            raise ValueError(f"Invalid entry format: {entry}")

        match_found = any(
            difflib.SequenceMatcher(None, ground_truth, pred).ratio() > 0.8
            for pred in predictions[:k]
        )

        if match_found:
            correct_topk += 1

        if predictions and ground_truth.strip().lower() == predictions[0].strip().lower():
            exact_matches += 1

    topk_accuracy = correct_topk / total if total else 0
    top1_exact = exact_matches / total if total else 0

    print(f"\n🔍 Evaluation Results:")
    print(f"→ Top-{k} accuracy (fuzzy): {topk_accuracy:.2%}")
    print(f"→ Top-1 exact match:        {top1_exact:.2%}")


In [55]:
evaluate_model(eval_data, k=5)


ValueError: Invalid entry format: query

In [56]:
import pandas as pd
import difflib

def safe_str(val):
    """Safely convert to string, handling NaNs."""
    if pd.isna(val):
        return ""
    return str(val)

def evaluate_model(eval_df, k=5):
    """
    Evaluate model performance from a DataFrame with columns:
    ['query', 'ground_truth', 'predictions']

    Parameters:
        eval_df (pd.DataFrame): DataFrame with evaluation data.
        k (int): Top-k to consider for fuzzy matching.
    """
    # Convert DataFrame rows to dictionaries
    eval_data = eval_df.to_dict(orient="records")

    total = len(eval_data)
    correct_topk = 0
    exact_matches = 0

    for entry in eval_data:
        query = safe_str(entry.get("query", ""))
        ground_truth = safe_str(entry.get("ground_truth", ""))
        predictions = entry.get("predictions", [])

        # Ensure predictions is a list of strings
        predictions = [safe_str(p) for p in predictions if p]

        # Top-k fuzzy match (e.g. > 80% similarity)
        match_found = any(
            difflib.SequenceMatcher(None, ground_truth, pred).ratio() > 0.8
            for pred in predictions[:k]
        )

        if match_found:
            correct_topk += 1

        # Top-1 exact match
        if predictions and ground_truth.strip().lower() == predictions[0].strip().lower():
            exact_matches += 1

    # Final metrics
    topk_accuracy = correct_topk / total if total else 0
    top1_exact = exact_matches / total if total else 0

    print("\n🔍 Evaluation Results:")
    print(f"→ Top-{k} fuzzy match accuracy: {topk_accuracy:.2%}")
    print(f"→ Top-1 exact match accuracy:   {top1_exact:.2%}")


In [57]:
import pandas as pd
import difflib

def safe_str(val):
    """Convert value to string safely."""
    if pd.isna(val):
        return ""
    return str(val)

def check_accuracy(df, k=5):
    """
    Check accuracy of model predictions.

    Parameters:
        df (pd.DataFrame): must have 'ground_truth' and 'predictions' columns
        k (int): top-k predictions to consider for fuzzy match
    """
    total = len(df)
    top_k_matches = 0
    top_1_exact_matches = 0

    for _, row in df.iterrows():
        ground_truth = safe_str(row["ground_truth"])
        predictions = [safe_str(p) for p in row["predictions"]]

        # Top-k fuzzy match (using difflib)
        for pred in predictions[:k]:
            similarity = difflib.SequenceMatcher(None, ground_truth.lower(), pred.lower()).ratio()
            if similarity > 0.8:
                top_k_matches += 1
                break

        # Top-1 exact match
        if predictions and predictions[0].strip().lower() == ground_truth.strip().lower():
            top_1_exact_matches += 1

    top_k_accuracy = top_k_matches / total if total else 0
    top_1_accuracy = top_1_exact_matches / total if total else 0

    print("\n✅ Accuracy Results:")
    print(f"🎯 Top-{k} Fuzzy Match Accuracy: {top_k_accuracy:.2%}")
    print(f"✅ Top-1 Exact Match Accuracy:   {top_1_accuracy:.2%}")


In [59]:
import pandas as pd
import numpy as np
import difflib
from sentence_transformers import SentenceTransformer

# -------------------------
# Step 1: Load Evaluation Data
# -------------------------
eval_data = pd.read_json("Dataset Queries-Securities for Model Creation.csv.json")

# -------------------------
# Step 2: Embed Queries and Search FAISS
# -------------------------
def get_top_k_predictions(query, k=5):
    query_vector = model.encode([query], normalize_embeddings=True)
    scores, indices = faiss_index.search(np.array(query_vector), k)
    return [fund_id_to_data[i]["fund_name"] for i in indices[0]]

eval_data["predictions"] = eval_data["query"].apply(lambda q: get_top_k_predictions(q, k=5))

# -------------------------
# Step 3: Define Accuracy Function
# -------------------------
def safe_str(val):
    if pd.isna(val):
        return ""
    return str(val)

def check_accuracy(df, k=5, verbose=False):
    if df.empty or "ground_truth" not in df.columns or "predictions" not in df.columns:
        print("❌ Dataset is missing required columns.")
        return

    total = len(df)
    top_k_matches = 0
    top_1_matches = 0

    for idx, row in df.iterrows():
        ground_truth = safe_str(row["ground_truth"])
        predictions = row["predictions"]
        predictions = [safe_str(p) for p in predictions]

        # Top-k fuzzy match
        fuzzy_match = any(
            difflib.SequenceMatcher(None, ground_truth.lower(), pred.lower()).ratio() > 0.8
            for pred in predictions[:k]
        )
        if fuzzy_match:
            top_k_matches += 1

        # Top-1 exact match
        if predictions and predictions[0].strip().lower() == ground_truth.strip().lower():
            top_1_matches += 1

        if verbose:
            print(f"\nQuery: {row['query']}")
            print(f"GT: {ground_truth}")
            print(f"Predictions: {predictions}")
            print(f"Top-1 Match: {predictions[0] == ground_truth}")
            print(f"Top-k Fuzzy Match: {fuzzy_match}")

    print("\n✅ Evaluation Results")
    print(f"🎯 Top-{k} Fuzzy Match Accuracy: {top_k_matches / total:.2%}")
    print(f"✅ Top-1 Exact Match Accuracy:   {top_1_matches / total:.2%}")

# -------------------------
# Step 4: Run Evaluation
# -------------------------
check_accuracy(eval_data, k=5, verbose=True)


FileNotFoundError: File Dataset Queries-Securities for Model Creation.csv.json does not exist

In [61]:
eval_data["predictions"] = eval_data["query"].apply(lambda q: get_top_k_predictions(q, k=5))
check_accuracy(eval_data, k=5, verbose=True)


NameError: name 'get_top_k_predictions' is not defined

In [62]:
import pandas as pd
import numpy as np
import difflib
from sentence_transformers import SentenceTransformer
import faiss

# Load your model and FAISS index (adjust if already loaded)
model = SentenceTransformer("BAAI/bge-small-en")

# Assuming you already have this:
# funds_df = pd.read_json("mutual_funds_data.json", lines=True)
fund_texts = funds_df["description"].tolist()
fund_vectors = model.encode(fund_texts, normalize_embeddings=True)

index = faiss.IndexFlatIP(fund_vectors.shape[1])
index.add(np.array(fund_vectors))

fund_id_to_data = dict(enumerate(funds_df.to_dict(orient="records")))

# 🔁 Function to get top-k fund predictions
def get_top_k_predictions(query, k=5):
    query_vector = model.encode([query], normalize_embeddings=True)
    scores, indices = index.search(np.array(query_vector), k)
    return [fund_id_to_data[i]["fund_name"] for i in indices[0]]

# ✅ Safe string conversion
def safe_str(val):
    if pd.isna(val):
        return ""
    return str(val)

# 🧠 Accuracy evaluation
def check_accuracy(df, k=5, verbose=False):
    if df.empty or "ground_truth" not in df.columns or "predictions" not in df.columns:
        print("❌ Dataset is missing required columns.")
        return

    total = len(df)
    top_k_matches = 0
    top_1_matches = 0

    for idx, row in df.iterrows():
        ground_truth = safe_str(row["ground_truth"])
        predictions = row["predictions"]
        predictions = [safe_str(p) for p in predictions]

        fuzzy_match = any(
            difflib.SequenceMatcher(None, ground_truth.lower(), pred.lower()).ratio() > 0.8
            for pred in predictions[:k]
        )
        if fuzzy_match:
            top_k_matches += 1

        if predictions and predictions[0].strip().lower() == ground_truth.strip().lower():
            top_1_matches += 1

        if verbose:
            print(f"\nQuery: {row['query']}")
            print(f"GT: {ground_truth}")
            print(f"Predictions: {predictions}")
            print(f"Top-1 Match: {predictions[0] == ground_truth}")
            print(f"Top-k Fuzzy Match: {fuzzy_match}")

    print("\n✅ Evaluation Results")
    print(f"🎯 Top-{k} Fuzzy Match Accuracy: {top_k_matches / total:.2%}")
    print(f"✅ Top-1 Exact Match Accuracy:   {top_1_matches / total:.2%}")

# 🗂️ Load the evaluation dataset (replace with your actual file path)
eval_data = pd.read_csv("Dataset Queries-Securities for Model Creation.csv")

# 👀 Check columns and rename if needed
print(eval_data.columns)

# If column names are something like 'Query' and 'Answer', rename:
eval_data.rename(columns={"Query": "query", "Answer": "ground_truth"}, inplace=True)

# 🔮 Generate predictions
eval_data["predictions"] = eval_data["query"].apply(lambda q: get_top_k_predictions(q, k=5))

# ✅ Evaluate
check_accuracy(eval_data, k=5, verbose=True)


KeyError: 'description'

In [63]:
print(funds_df.columns)


Index(['schemeCode', 'amfiCode', 'amcCode', 'schemeName', 'typeCode',
       'optCode', 'classCode', 'plan', 'etf', 'stp',
       ...
       'expenseRatio', 'sectorExposure', 'schiskOMeter', 'schbenchmarkTier1',
       'schemebenchmarkTier1', '5YVolatilibsite', 'sory', 'amcCtinoRatio',
       '10YVolatilitode', 'benchminRedemptionAmount'],
      dtype='object', length=114)


In [66]:
funds_df["sory"].head(10)


,sory
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [60]:
eval_data = pd.read_csv("Dataset Queries-Securities for Model Creation.csv")


In [67]:
fund_texts = funds_df["schemeName"].astype(str).tolist()


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Use schemeName or sory depending on which has better descriptions
fund_texts = funds_df["schemeName"].astype(str).tolist()

model = SentenceTransformer("BAAI/bge-small-en")
fund_vectors = model.encode(fund_texts, normalize_embeddings=True)

index = faiss.IndexFlatIP(fund_vectors.shape[1])
index.add(np.array(fund_vectors))

fund_id_to_data = dict(enumerate(funds_df.to_dict(orient="records")))


In [60]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [61]:
%%writefile app.py
import streamlit as st

st.title("📊 Mutual Fund Finder")
query = st.text_input("Enter your query")

if query:
    st.write(f"🔍 You searched for: {query}")
    # Here, you would call your backend pipeline and display results


Writing app.py


In [62]:
from pyngrok import ngrok

# Kill any previous tunnels
!pkill streamlit

# Open a tunnel
public_url = ngrok.connect(port=8501)
print("🚀 Streamlit is running at:", public_url)

# Run Streamlit
!streamlit run app.py &>/content/logs.txt &


ERROR:pyngrok.process.ngrok:t=2025-04-11T21:54:18+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-11T21:54:18+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-11T21:54:18+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [65]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2vbKxAr483RM5WyKMjsmx1alLoh_4KNkDLRNd3mty1TnF5v3b")

# Start the tunnel to localhost:8501
public_url = ngrok.connect(addr="8501")
print("🚀 Streamlit app running at:", public_url)

# Start your Streamlit app (make sure app.py is your entry point)
!streamlit run app.py &>/content/logs.txt &


🚀 Streamlit app running at: NgrokTunnel: "https://0fae-34-125-252-39.ngrok-free.app" -> "http://localhost:8501"


In [66]:
import streamlit as st
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss

# Load model & FAISS (adjust paths as per your setup)
model = SentenceTransformer("BAAI/bge-small-en")
funds_df = pd.read_json("mutual_funds_data.json", lines=True)

# Load your FAISS index and vectors
# faiss_index = faiss.read_index("your_index_file.index")  # Optional if you're using FAISS

st.title("💰 Mutual Fund Search Assistant")
query = st.text_input("Ask a query about mutual funds")

if query:
    with st.spinner("Searching..."):
        # Embed and search
        query_vec = model.encode(query, normalize_embeddings=True)

        # Replace below with your pipeline logic
        results = funds_df[funds_df["schemeName"].str.contains("HDFC", case=False)]  # Example dummy logic

        # Show results
        st.write("### 🔍 Top Results:")
        st.dataframe(results[["schemeName", "fundPrimarySector", "expenseRatio"]].head(5))


2025-04-11 22:07:21.007 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:07:21.345 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-11 22:07:21.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:07:21.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:07:21.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:07:21.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:07:21.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:07:21.370 Session state does not 

In [67]:
!streamlit run app.py &>/content/logs.txt &
from pyngrok import ngrok
ngrok.set_auth_token("2vbKxAr483RM5WyKMjsmx1alLoh_4KNkDLRNd3mty1TnF5v3b")
public_url = ngrok.connect(addr="8501")
print("🚀 Streamlit app running at:", public_url)


🚀 Streamlit app running at: NgrokTunnel: "https://ec5a-34-125-252-39.ngrok-free.app" -> "http://localhost:8501"


In [69]:
# app.py

import streamlit as st
import pandas as pd
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer

# Load data
funds_df = pd.read_json("mutual_funds_data.json", lines=True)

# Load model
model = SentenceTransformer("BAAI/bge-small-en")

# Load spaCy for NER
nlp = spacy.load("en_core_web_sm")

# Entity & keyword extraction
def extract_entities_and_keywords(query):
    doc = nlp(query)
    keywords = {}

    for ent in doc.ents:
        if ent.label_ in ["ORG", "PRODUCT"]:
            keywords["schemeName"] = ent.text
        if ent.label_ == "PERCENT" and "expense" in query.lower():
            try:
                percent_val = float(ent.text.replace("%", "").strip())
                keywords["expenseRatio"] = percent_val
            except:
                pass

    q = query.lower()

    if "technology" in q or "tech" in q or "it" in q:
        keywords["fundPrimarySector"] = "Technology"
    if "infrastructure" in q or "infra" in q:
        keywords["fundPrimarySector"] = "Infrastructure"
    if "tax" in q:
        keywords["classCode"] = "ELSS"
    if "low expense" in q or "cheap" in q:
        keywords["expenseRatio"] = "low"

    return keywords

# Main retrieval pipeline
def full_fund_search_pipeline(query, k=5):
    query_vector = model.encode("Represent this sentence for retrieval: " + query, normalize_embeddings=True)

    fund_texts = funds_df["schemeName"].astype(str).tolist()
    fund_vectors = model.encode(fund_texts, normalize_embeddings=True)

    scores = np.dot(fund_vectors, query_vector)
    top_k_indices = np.argsort(scores)[-k:][::-1]

    top_funds = funds_df.iloc[top_k_indices].copy()
    top_funds["score"] = scores[top_k_indices]

    # Apply rule-based keyword boosting
    keywords = extract_entities_and_keywords(query)
    top_funds["score_boost"] = 0

    for key, value in keywords.items():
        if key in top_funds.columns:
            if key == "expenseRatio" and value == "low":
                top_funds["score_boost"] -= top_funds["expenseRatio"].fillna(1.0)
            else:
                top_funds["score_boost"] += top_funds[key].astype(str).str.contains(str(value), case=False, na=False).astype(int)

    top_funds["final_score"] = top_funds["score"] + top_funds["score_boost"]
    top_funds = top_funds.sort_values(by="final_score", ascending=False)

    return top_funds.head(k)

# --- Streamlit App UI ---
st.set_page_config(page_title="Mutual Fund Finder", layout="centered")
st.title("📊 Mutual Fund Finder")
query = st.text_input("Enter your query")

if query:
    st.write(f"🔍 You searched for: `{query}`")
    with st.spinner("Searching..."):
        results = full_fund_search_pipeline(query)
        st.success("Results Ready!")
        st.dataframe(results[["schemeName", "fundPrimarySector", "expenseRatio", "final_score"]])


2025-04-11 22:12:21.400 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:12:21.402 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:12:21.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:12:21.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:12:21.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:12:21.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:12:21.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 22:12:21.409 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [73]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer

# Load data
funds_df = pd.read_json("mutual_funds_data.json", lines=True)

# Load model
model = SentenceTransformer("BAAI/bge-small-en")

# Load spaCy for NER
nlp = spacy.load("en_core_web_sm")

# Entity & keyword extraction
def extract_entities_and_keywords(query):
    doc = nlp(query)
    keywords = {}

    for ent in doc.ents:
        if ent.label_ in ["ORG", "PRODUCT"]:
            keywords["schemeName"] = ent.text
        if ent.label_ == "PERCENT" and "expense" in query.lower():
            try:
                percent_val = float(ent.text.replace("%", "").strip())
                keywords["expenseRatio"] = percent_val
            except:
                pass

    q = query.lower()

    if "technology" in q or "tech" in q or "it" in q:
        keywords["fundPrimarySector"] = "Technology"
    if "infrastructure" in q or "infra" in q:
        keywords["fundPrimarySector"] = "Infrastructure"
    if "tax" in q:
        keywords["classCode"] = "ELSS"
    if "low expense" in q or "cheap" in q:
        keywords["expenseRatio"] = "low"

    return keywords

# Main retrieval pipeline
def full_fund_search_pipeline(query, k=5):
    query_vector = model.encode("Represent this sentence for retrieval: " + query, normalize_embeddings=True)
    fund_texts = funds_df["schemeName"].astype(str).tolist()
    fund_vectors = model.encode(fund_texts, normalize_embeddings=True)
    scores = np.dot(fund_vectors, query_vector)
    top_k_indices = np.argsort(scores)[-k:][::-1]
    top_funds = funds_df.iloc[top_k_indices].copy()
    top_funds["score"] = scores[top_k_indices]

    # Keyword boosting
    keywords = extract_entities_and_keywords(query)
    top_funds["score_boost"] = 0

    for key, value in keywords.items():
        if key in top_funds.columns:
            if key == "expenseRatio" and value == "low":
                top_funds["score_boost"] -= top_funds["expenseRatio"].fillna(1.0)
            else:
                top_funds["score_boost"] += top_funds[key].astype(str).str.contains(str(value), case=False, na=False).astype(int)

    top_funds["final_score"] = top_funds["score"] + top_funds["score_boost"]
    top_funds = top_funds.sort_values(by="final_score", ascending=False)
    return top_funds.head(k)

# Streamlit UI
st.set_page_config(page_title="Mutual Fund Finder", layout="centered")
st.title("📊 Mutual Fund Finder")
query = st.text_input("Enter your query")

if query:
    st.write(f"🔍 You searched for: `{query}`")
    with st.spinner("Searching..."):
        results = full_fund_search_pipeline(query)
        st.success("Results Ready!")
        st.dataframe(results[["schemeName", "fundPrimarySector", "expenseRatio", "final_score"]])


Overwriting app.py


In [75]:
import time
!streamlit run app.py &
time.sleep(10)
public_url = ngrok.connect(8501)
print("Streamlit App URL:", public_url)





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.252.39:8501

  Stopping...
Streamlit App URL: NgrokTunnel: "https://d035-34-125-252-39.ngrok-free.app" -> "http://localhost:8501"


ModuleNotFoundError: No module named 'gradio'

In [77]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00




---



ValueError: Unexpected character found when decoding object value

KeyboardInterrupt: 

In [9]:
evaluate_model(eval_data, k=5)

NameError: name 'evaluate_model' is not defined